In [1]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Load data
# Distance matrix
clinic_supplier_distance = pd.read_csv('clinic_supplier_distance_matrix.csv', index_col=0)

# Material costs
price_matrix = pd.read_csv('price_matrix.csv', index_col=0)


# Demand projections
demand_projection = pd.read_csv('demand_projection_matrix.csv', index_col=0)

# Bill of Materials (BoM)
bom = pd.read_csv('bom.csv')

# Fixed cost per unit distance
cost_per_unit_distance = 0.0005

# Step 1: Process demand projections with BoM to calculate material demand (D_{i,k})
# Expand treatment demands into material demands
material_demand = pd.DataFrame()
for clinic in demand_projection.index:
    clinic_demand = []
    for treatment, quantity in demand_projection.loc[clinic].items():
        treatment_materials = bom[bom['Code'] == treatment]
        for _, row in treatment_materials.iterrows():
            material = row['Material']
            required_quantity = row['Quantity'] * quantity
            clinic_demand.append({'Clinic': clinic, 'Material': material, 'Demand': required_quantity})
    clinic_demand_df = pd.DataFrame(clinic_demand)
    material_demand = pd.concat([material_demand, clinic_demand_df], ignore_index=True)

# Aggregate material demands per clinic
material_demand = material_demand.groupby(['Clinic', 'Material'])['Demand'].sum().reset_index()

# Step 2: Define the Pyomo model
model = pyo.ConcreteModel()

# Sets
model.clinics = pyo.Set(initialize=material_demand['Clinic'].unique())
model.suppliers = pyo.Set(initialize=clinic_supplier_distance.columns)
model.materials = pyo.Set(initialize=material_demand['Material'].unique())

# Parameters
# Demand (D_{i,k})
material_demand_dict = material_demand.set_index(['Clinic', 'Material'])['Demand'].to_dict()
model.demand = pyo.Param(model.clinics, model.materials, initialize=material_demand_dict, default=0)

# Unit cost (C_{j,k})
# Transpose the price_matrix for stacking in the correct format
price_matrix_t = price_matrix.T
unit_cost_dict = price_matrix_t.stack().to_dict()

# Ensure suppliers and materials are explicitly indexed
suppliers = price_matrix.columns.tolist()
materials = price_matrix.index.tolist()

# Create a dictionary for unit costs with the correct key order
unit_cost_dict_corrected = {
    (supplier, material): price_matrix.loc[material, supplier]
    for supplier in suppliers
    for material in materials
}

# Initialize the Pyomo parameter with the corrected dictionary
model.unit_cost = pyo.Param(model.suppliers, model.materials, initialize=unit_cost_dict_corrected, default=float('inf'))

# Distance (T_{i,j})
distance_dict = clinic_supplier_distance.stack().to_dict()
model.distance = pyo.Param(model.clinics, model.suppliers, initialize=distance_dict, default=float('inf'))

# Cost per unit distance
model.cost_per_unit_distance = pyo.Param(initialize=cost_per_unit_distance)

# Capacity (U_{j,k}) - Set to high values for now
high_capacity = 1e6
model.capacity = pyo.Param(model.suppliers, model.materials, initialize=lambda model, j, k: high_capacity)

# Decision Variables
model.x = pyo.Var(model.clinics, model.suppliers, model.materials, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(
        model.x[i, j, k] * (
            model.unit_cost[j, k] + model.cost_per_unit_distance * model.distance[i, j]
        ) * model.demand[i, k]
        for i in model.clinics for j in model.suppliers for k in model.materials
    )
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Demand fulfillment
def demand_fulfillment_rule(model, i, k):
    return sum(model.x[i, j, k] for j in model.suppliers) == 1
model.demand_fulfillment = pyo.Constraint(model.clinics, model.materials, rule=demand_fulfillment_rule)

# Capacity constraints
def capacity_rule(model, j, k):
    return sum(model.x[i, j, k] * model.demand[i, k] for i in model.clinics) <= model.capacity[j, k]
model.capacity_constraint = pyo.Constraint(model.suppliers, model.materials, rule=capacity_rule)

# Solve the model
opt = SolverFactory('cbc', executable='C:\\bin\\cbc.exe')
results = opt.solve(model, tee=True)

# Extract results
allocation = []
for i in model.clinics:
    for j in model.suppliers:
        for k in model.materials:
            if pyo.value(model.x[i, j, k]) > 0.5:
                allocation.append({'Clinic': i, 'Supplier': j, 'Material': k})

allocation_df = pd.DataFrame(allocation)
allocation_df.to_csv('allocation_results.csv', index=False)



Welcome to the CBC MILP Solver 
Version: 2.9.7 
Build Date: Oct 10 2015 
Revision Number: 2226 

command line - C:\bin\cbc.exe -printingOptions all -import C:\Users\Zak\AppData\Local\Temp\tmp_ua0kf9i.pyomo.lp -stat=1 -solve -solu C:\Users\Zak\AppData\Local\Temp\tmp_ua0kf9i.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 1500 (-600) rows, 30000 (0) columns and 30000 (-30000) elements
Statistics for presolved model
Original problem has 30000 integers (30000 of which binary)
==== 0 zero objective 29985 different
==== absolute objective values 29985 different
==== for integers 0 zero objective 29985 different
==== for integers absolute objective values 29985 different
===== end objective counts


Problem has 1500 rows, 30000 columns (30000 with objective) and 30000 elements
There are 30000 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 


In [ ]:
# Calculate total unit cost and transportation cost
unit_cost_total = sum(
    pyo.value(model.x[i, j, k]) * model.unit_cost[j, k] * model.demand[i, k]
    for i in model.clinics for j in model.suppliers for k in model.materials
)

transportation_cost_total = sum(
    pyo.value(model.x[i, j, k]) * model.cost_per_unit_distance * model.distance[i, j] * model.demand[i, k]
    for i in model.clinics for j in model.suppliers for k in model.materials
)

overall_cost_total = pyo.value(model.objective)

print(f"Total Unit Cost: {unit_cost_total:,.0f}")
print(f"Total Transportation Cost: {transportation_cost_total:,.0f}")

objective_value = pyo.value(model.objective)
print(f'Objective value: {objective_value:,.0f}')


Total Unit Cost: 3,783,043
Total Transportation Cost: 1,186,131
Objective value: 4,969,174


In [4]:
allocation_df

,Clinic,Supplier,Material
0,N0001,Supplier 16,Material 1
1,N0001,Supplier 16,Material 11
2,N0001,Supplier 16,Material 13
3,N0001,Supplier 16,Material 16
4,N0001,Supplier 16,Material 17
...,...,...,...
1495,V3340,Supplier 15,Material 30
1496,V3340,Supplier 15,Material 4
1497,V3340,Supplier 15,Material 7
1498,V3340,Supplier 15,Material 8


In [5]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 1115259878.2924988
  Upper bound: 1115259878.2924988
  Number of objectives: 1
  Number of constraints: 1680
  Number of variables: 30000
  Number of binary variables: 30000
  Number of integer variables: 30000
  Number of nonzeros: 30000
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 3.76
  Wallclock time: 3.76
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 4.038940191268921
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [15]:
# model.pprint()



Objective value: 1,115,259,878


### TRIAL for DEMO (Added Opportunity Gain)



In [18]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

class ModelSupplierSelection:
    def __init__(self, clinic_supplier_distance, price_matrix, demand_projection, bom, cost_per_unit_distance=0.0005):
        self.clinic_supplier_distance = clinic_supplier_distance
        self.price_matrix = price_matrix
        self.demand_projection = demand_projection
        self.bom = bom
        self.cost_per_unit_distance = cost_per_unit_distance

        self.material_demand = self._process_demand_projections()
        self.model = self._build_model()

    def _process_demand_projections(self):
        material_demand = pd.DataFrame()
        for clinic in self.demand_projection.index:
            clinic_demand = []
            for treatment, quantity in self.demand_projection.loc[clinic].items():
                treatment_materials = self.bom[self.bom['Code'] == treatment]
                for _, row in treatment_materials.iterrows():
                    material = row['Material']
                    required_quantity = row['Quantity'] * quantity
                    clinic_demand.append({'Clinic': clinic, 'Material': material, 'Demand': required_quantity})
            clinic_demand_df = pd.DataFrame(clinic_demand)
            material_demand = pd.concat([material_demand, clinic_demand_df], ignore_index=True)

        # Aggregate material demands per clinic
        return material_demand.groupby(['Clinic', 'Material'])['Demand'].sum().reset_index()

    def _build_model(self):
        model = pyo.ConcreteModel()

        # Sets
        model.clinics = pyo.Set(initialize=self.material_demand['Clinic'].unique())
        model.suppliers = pyo.Set(initialize=self.clinic_supplier_distance.columns)
        model.materials = pyo.Set(initialize=self.material_demand['Material'].unique())

        # Parameters
        # Demand (D_{i,k})
        material_demand_dict = self.material_demand.set_index(['Clinic', 'Material'])['Demand'].to_dict()
        model.demand = pyo.Param(model.clinics, model.materials, initialize=material_demand_dict, default=0)

        # Unit cost (C_{j,k})
        suppliers = self.price_matrix.columns.tolist()
        materials = self.price_matrix.index.tolist()
        unit_cost_dict_corrected = {
            (supplier, material): self.price_matrix.loc[material, supplier]
            for supplier in suppliers
            for material in materials
        }
        model.unit_cost = pyo.Param(model.suppliers, model.materials, initialize=unit_cost_dict_corrected, default=float('inf'))

        # Distance (T_{i,j})
        distance_dict = self.clinic_supplier_distance.stack().to_dict()
        model.distance = pyo.Param(model.clinics, model.suppliers, initialize=distance_dict, default=float('inf'))

        # Cost per unit distance
        model.cost_per_unit_distance = pyo.Param(initialize=self.cost_per_unit_distance)

        # Capacity (U_{j,k}) - Set to high values for now
        high_capacity = 1e6
        model.capacity = pyo.Param(model.suppliers, model.materials, initialize=lambda model, j, k: high_capacity)

        # Decision Variables
        model.x = pyo.Var(model.clinics, model.suppliers, model.materials, within=pyo.Binary)

        # Objective Function
        def objective_rule(model):
            return sum(
                model.x[i, j, k] * (
                    model.unit_cost[j, k] + model.cost_per_unit_distance * model.distance[i, j]
                ) * model.demand[i, k]
                for i in model.clinics for j in model.suppliers for k in model.materials
            )
        model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

        # Constraints
        # Demand fulfillment
        def demand_fulfillment_rule(model, i, k):
            return sum(model.x[i, j, k] for j in model.suppliers) == 1
        model.demand_fulfillment = pyo.Constraint(model.clinics, model.materials, rule=demand_fulfillment_rule)

        # Capacity constraints
        def capacity_rule(model, j, k):
            return sum(model.x[i, j, k] * model.demand[i, k] for i in model.clinics) <= model.capacity[j, k]
        model.capacity_constraint = pyo.Constraint(model.suppliers, model.materials, rule=capacity_rule)

        return model

    def solve(self, solver_path='C:\\bin\\cbc.exe'):
        opt = SolverFactory('cbc', executable=solver_path)
        results = opt.solve(self.model, tee=True)

        # Extract results
        allocation = []
        for i in self.model.clinics:
            for j in self.model.suppliers:
                for k in self.model.materials:
                    if pyo.value(self.model.x[i, j, k]) > 0.5:
                        allocation.append({'Clinic': i, 'Supplier': j, 'Material': k})

        allocation_df = pd.DataFrame(allocation)
        allocation_df.to_csv('allocation_results.csv', index=False)
        return allocation_df

    def calculate_objective_value(self, allocation_df):
        # Calculate objective value for a given allocation
        total_cost = 0
        for _, row in allocation_df.iterrows():
            clinic = row['Clinic']
            supplier = row['Supplier']
            material = row['Material']

            # Filter material_demand directly instead of using query
            demand_value = self.material_demand[
                (self.material_demand['Clinic'] == clinic) & (self.material_demand['Material'] == material)
            ]['Demand'].values[0]

            # Calculate cost for this allocation
            total_cost += (
                self.price_matrix.loc[material, supplier] +
                self.cost_per_unit_distance * self.clinic_supplier_distance.loc[clinic, supplier]
            ) * demand_value
        return total_cost

    def comparison_to_previous_optimal(self, last_optimal_configuration=None):
        
        if last_optimal_configuration is None:
            new_allocation = self.solve()
            
            return {
                'Baseline Best Solution': None,
                'New Solution Value': self.calculate_objective_value(new_allocation),
                'Opportunity Gain': None,
                'Allocation': new_allocation}
                
        else:
        
            # Calculate baseline best solution objective value
            baseline_best_solution = self.calculate_objective_value(last_optimal_configuration)

            # Solve for the current optimal solution
            new_allocation = self.solve()
            new_solution_value = self.calculate_objective_value(new_allocation)

            # Calculate opportunity gain
            opportunity_gain = baseline_best_solution - new_solution_value


            
            return {
                'Baseline Best Solution': baseline_best_solution,
                'New Solution Value': new_solution_value,
                'Opportunity Gain': opportunity_gain,
                'Allocation': new_allocation if opportunity_gain > 0 else None
            }

    def get_cost_breakdown(self):
        unit_cost_total = sum(
            pyo.value(self.model.x[i, j, k]) * self.model.unit_cost[j, k] * self.model.demand[i, k]
            for i in self.model.clinics for j in self.model.suppliers for k in self.model.materials
        )

        transportation_cost_total = sum(
            pyo.value(self.model.x[i, j, k]) * self.model.cost_per_unit_distance * self.model.distance[i, j] * self.model.demand[i, k]
            for i in self.model.clinics for j in self.model.suppliers for k in self.model.materials
        )

        overall_cost_total = pyo.value(self.model.objective)

        return {
            'Unit Cost Total': unit_cost_total,
            'Transportation Cost Total': transportation_cost_total,
            'Overall Cost Total': overall_cost_total
        }




In [23]:
# Usage example (replace file paths with actual paths)
clinic_supplier_distance = pd.read_csv('clinic_supplier_distance_matrix.csv', index_col=0)
price_matrix = pd.read_csv('price_matrix.csv', index_col=0)
demand_projection = pd.read_csv('demand_projection_matrix.csv', index_col=0)
bom = pd.read_csv('bom.csv')

model = ModelSupplierSelection(clinic_supplier_distance, price_matrix, demand_projection, bom)
allocation_results = model.solve()


Welcome to the CBC MILP Solver 
Version: 2.9.7 
Build Date: Oct 10 2015 
Revision Number: 2226 

command line - C:\bin\cbc.exe -printingOptions all -import C:\Users\Zak\AppData\Local\Temp\tmpo26dvx6o.pyomo.lp -stat=1 -solve -solu C:\Users\Zak\AppData\Local\Temp\tmpo26dvx6o.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 1500 (-600) rows, 30000 (0) columns and 30000 (-30000) elements
Statistics for presolved model
Original problem has 30000 integers (30000 of which binary)
==== 0 zero objective 29985 different
==== absolute objective values 29985 different
==== for integers 0 zero objective 29985 different
==== for integers absolute objective values 29985 different
===== end objective counts


Problem has 1500 rows, 30000 columns (30000 with objective) and 30000 elements
There are 30000 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 


In [20]:
# allocation_results.to_csv('last_optimal_allocation.csv', index=False)

In [24]:
last_optimal = pd.read_csv('last_optimal_allocation.csv')
last_optimal

,Clinic,Supplier,Material
0,N0001,Supplier 1,Material 17
1,N0001,Supplier 1,Material 24
2,N0001,Supplier 1,Material 25
3,N0001,Supplier 1,Material 27
4,N0001,Supplier 2,Material 30
...,...,...,...
1495,V3340,Supplier 10,Material 13
1496,V3340,Supplier 10,Material 19
1497,V3340,Supplier 10,Material 27
1498,V3340,Supplier 10,Material 29


In [25]:
# last_optimal = pd.read_csv('last_optimal_allocation.csv')
comparison = model.comparison_to_previous_optimal(last_optimal)
comparison

Welcome to the CBC MILP Solver 
Version: 2.9.7 
Build Date: Oct 10 2015 
Revision Number: 2226 

command line - C:\bin\cbc.exe -printingOptions all -import C:\Users\Zak\AppData\Local\Temp\tmpuds6jgey.pyomo.lp -stat=1 -solve -solu C:\Users\Zak\AppData\Local\Temp\tmpuds6jgey.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 1500 (-600) rows, 30000 (0) columns and 30000 (-30000) elements
Statistics for presolved model
Original problem has 30000 integers (30000 of which binary)
==== 0 zero objective 29985 different
==== absolute objective values 29985 different
==== for integers 0 zero objective 29985 different
==== for integers absolute objective values 29985 different
===== end objective counts


Problem has 1500 rows, 30000 columns (30000 with objective) and 30000 elements
There are 30000 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 


{'Baseline Best Solution': 5865157.447060003,
 'New Solution Value': 4969174.488790009,
 'Opportunity Gain': 895982.9582699938}